# Experiment with writing dataset class for LVIS data

In [4]:
from pathlib import Path
import numpy as np
import torch
from PIL import Image


class LvisDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms, stage='train'):
        self.root = Path(root)
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = sorted((self.root/'images/{}2017'.format(stage)).glob('*'))
        self.mask_file = self.root/('annotations/lvis_v1_{}.json'.format(stage))

    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)
        # convert the PIL Image into a numpy array
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [5]:
root = Path('../Datasets/coco/')
stage = 'train'

In [9]:
sorted((root/('annotations/lvis_v1_{}.json'.format(stage))).glob('*'))

[]

In [10]:
root/('annotations/lvis_v1_{}.json'.format(stage))

PosixPath('../Datasets/coco/annotations/lvis_v1_train.json')

In [14]:
sorted((root/'images/{}2017'.format(stage)).glob('*'))

[PosixPath('../Datasets/coco/images/train2017/000000000009.jpg'),
 PosixPath('../Datasets/coco/images/train2017/000000000025.jpg'),
 PosixPath('../Datasets/coco/images/train2017/000000000030.jpg'),
 PosixPath('../Datasets/coco/images/train2017/000000000034.jpg'),
 PosixPath('../Datasets/coco/images/train2017/000000000036.jpg'),
 PosixPath('../Datasets/coco/images/train2017/000000000042.jpg'),
 PosixPath('../Datasets/coco/images/train2017/000000000049.jpg'),
 PosixPath('../Datasets/coco/images/train2017/000000000061.jpg'),
 PosixPath('../Datasets/coco/images/train2017/000000000064.jpg'),
 PosixPath('../Datasets/coco/images/train2017/000000000071.jpg'),
 PosixPath('../Datasets/coco/images/train2017/000000000072.jpg'),
 PosixPath('../Datasets/coco/images/train2017/000000000073.jpg'),
 PosixPath('../Datasets/coco/images/train2017/000000000074.jpg'),
 PosixPath('../Datasets/coco/images/train2017/000000000077.jpg'),
 PosixPath('../Datasets/coco/images/train2017/000000000078.jpg'),
 PosixPath